<a href="https://colab.research.google.com/github/biswajitmohanty/genai/blob/main/Simple_RNN_Predict_Tomorrows_Stock_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go

In [2]:
# Fetch Microsoft's stock data
ticker = "MSFT"
stock = yf.Ticker(ticker)
df = stock.history(period="5y")  # Get last 5 years of data

In [3]:
# Preprocessing: Scaling data
df = df[["Close"]]
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

In [4]:
# Create sequences for RNN
def create_sequences(data, time_steps=60):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(data[i+time_steps])
    return np.array(X), np.array(y)

In [5]:
time_steps = 60  # Use past 60 days to predict the next day
X, y = create_sequences(df_scaled, time_steps)

In [6]:
# Split data into training & test sets
X_train, X_test = X[:-1], X[-1:]
y_train, y_test = y[:-1], y[-1:]

In [7]:
# Build RNN model
model = Sequential([
    SimpleRNN(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    SimpleRNN(50, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=50, batch_size=16)

Epoch 1/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.1283
Epoch 2/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0120
Epoch 3/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0090
Epoch 4/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.0079
Epoch 5/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0077
Epoch 6/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0066
Epoch 7/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0061
Epoch 8/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0060
Epoch 9/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0059
Epoch 10/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0053
Epoch 11/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0044
Epoch 12/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0049
Epoch 13/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0048
Epoch 14/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0045
Epoch 15/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0045
Epoc

In [9]:
# Predict tomorrow's stock price
predicted_price_scaled = model.predict(X_test)
predicted_price = scaler.inverse_transform(predicted_price_scaled)[0, 0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step


In [10]:
print(f"\n Predicted closing price for {ticker} tomorrow: ${predicted_price:.2f}")

# Plot actual vs predicted
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index[-100:], y=df["Close"].iloc[-100:], mode='lines', name="Actual Price"))
fig.add_trace(go.Scatter(x=[df.index[-1], df.index[-1] + pd.Timedelta(days=1)],
                         y=[df["Close"].iloc[-1], predicted_price],
                         mode='lines+markers', name="Predicted Tomorrow"))
fig.update_layout(title=f"{ticker} Stock Price Prediction for Tomorrow",
                  xaxis_title="Date", yaxis_title="Price", template="plotly_dark")
fig.show()


 Predicted closing price for MSFT tomorrow: $393.56
